In [1]:
from selenium import webdriver 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
base_url = []

for i in range(1,6):
    if i == 1:
        url = 'https://www.airlinequality.com/lounge-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=100'
        base_url.append(url)
    else:
        url = f'https://www.airlinequality.com/lounge-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100'
        base_url.append(url)
base_url

['https://www.airlinequality.com/lounge-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/lounge-reviews/british-airways/page/1/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/lounge-reviews/british-airways/page/2/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/lounge-reviews/british-airways/page/3/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/lounge-reviews/british-airways/page/4/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/lounge-reviews/british-airways/page/5/?sortby=post_date%3ADesc&pagesize=100']

In [19]:
reviewer_name = []
review_sub = []
review_text = []
date_uploaded = []
airport_names = []
ratings_given = []
recommendations = []

for url in base_url:
    driver = webdriver.Chrome()
#     driver.maximize_window()
    driver.get(url)
    
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html5lib')
    
# finding reviewer name
    try:
        data = soup.find_all('span', attrs = {'itemprop':'name'})
        
        for dt in data:
            name = dt.get_text()
            reviewer_name.append(name)
    
    except Exception as e:
        name = 'Not Available'
        reviewer_name.append(name)
        print('Error finding element span:', e)
    
# find the heading of the review
    try:
        data = soup.find_all('h2', attrs = {'class':'text_header'})
        
        for dt in data:
            sub = dt.get_text()
            review_sub.append(sub)
        
    except Exception as e:
        sub = 'Not Available'
        review_sub.append(sub)
        print('Error finding element h2:', e)

# find the content of the review
    try:
        data = soup.find_all('div', attrs = {'class':'text_content'})
        
        for dt in data:
            review = dt.get_text()
            review_text.append(review)
        
    except Exception as e:
        review = 'Not Available'
        review_text.append(review)
        print('Error finding element div:', e)

# find the date review was uploaded
    try:
        data = soup.find_all('time', attrs = {'itemprop':'datePublished'})
        
        for dt in data:
            date = dt.get_text()
            date_uploaded.append(date)
            
    except Exception as e:
        date = np.nan
        date_uploaded.append(date)
        print('Error finding element time:', e)

# finding airport name and recommended or not
    try:
        temp_data = soup.find_all('div', attrs = {'class':'review-stats'})
        
        for data in temp_data:
            temp_dt = data.find('table', attrs = {'class':'review-ratings'})
            df = pd.read_html(str(temp_dt))[0]
            df = df.T
            cols = df.iloc[0]
            df = df[1:]
            df.columns = cols
            df.reset_index(drop=True, inplace=True)
            
            if 'Airport' in df.columns:
                airport = df['Airport'][0]
                airport_names.append(airport)
            else:
                airport_names.append(np.nan)
            
            if 'Recommended' in df.columns:
                reco = df['Recommended'][0]
                recommendations.append(reco)
            else:
                recommendations.append(np.nan)
            
    except Exception as e:
            print('Error finding element table:', e)

# finding the customer rating
    try:
        temp_data = soup.find_all('div', attrs = {'itemprop':'reviewRating', 'class':'rating-10'})
        
        for data in temp_data:
            dt = data.find('span', attrs = {'itemprop':'ratingValue'})
            rating = dt.get_text()
            ratings_given.append(rating)
    
    except Exception as e:
        ratings_given.append(np.nan)
        print('Error finding element div:', e)
        
            
driver.quit()


In [22]:
review_sub

['"there is not enough space"',
 '"Staff were welcoming and friendly"',
 '"rotten and totally incompetent airline"',
 '"BA staff who are rude, abrasive"',
 '"full service has been resumed"',
 '"the lounge is infested with mice"',
 '"cold and unwelcoming" ',
 '"not a bad experience"',
 '"shoddy level of service"',
 '"as basic as it gets"',
 '"Drink choices have been limited"',
 '"lack of breakfast on offer"',
 '"The food was surprisingly good"',
 '"experience could not have been better!',
 '"Way too crowded"',
 '"Overcrowded, inedible food"',
 '"British Airways Lounge is excellent"',
 '"5B lounge is so much better than the T5A lounge"',
 '"Very badly laid out lounge"',
 '"food offering has further declined"',
 '"Light, airy and decent food"',
 '"Rude staff and disgusting toilets"',
 '"very crowded and thus uncomfortable"',
 '"fantastically helpful staff"',
 '"food offering has declined dramatically"',
 '"lack of that truly luxurious feel"',
 '"lounge has no windows"',
 '"lounge was very

In [23]:
all_review_sub = []
for sub in review_sub:
    sub = sub.lower()
    sub = sub.replace('"','')
    sub = sub.replace('/','')
    sub = sub.strip()
    all_review_sub.append(sub)

In [24]:
all_review_sub

['there is not enough space',
 'staff were welcoming and friendly',
 'rotten and totally incompetent airline',
 'ba staff who are rude, abrasive',
 'full service has been resumed',
 'the lounge is infested with mice',
 'cold and unwelcoming',
 'not a bad experience',
 'shoddy level of service',
 'as basic as it gets',
 'drink choices have been limited',
 'lack of breakfast on offer',
 'the food was surprisingly good',
 'experience could not have been better!',
 'way too crowded',
 'overcrowded, inedible food',
 'british airways lounge is excellent',
 '5b lounge is so much better than the t5a lounge',
 'very badly laid out lounge',
 'food offering has further declined',
 'light, airy and decent food',
 'rude staff and disgusting toilets',
 'very crowded and thus uncomfortable',
 'fantastically helpful staff',
 'food offering has declined dramatically',
 'lack of that truly luxurious feel',
 'lounge has no windows',
 'lounge was very basic',
 'food was very hit and miss',
 'quality felt 

In [25]:
review_text

['✅ Trip Verified |  The lounge is clean but the seats are worn, and there is not enough space to accommodate all the passengers travelling in Club and those with silver and having access to the lounge. Nothing wrong, decent sandwiches but no hot food only soup, decent choice of alcoholic beverages and acceptable wines. Service with a smile. vOnly a shame the restrooms are extremely basic and not very clean.',
 '✅ Trip Verified |  The lounge is very spacious with a wide variety of areas to sit and a kids play area. Food was amazing with hot cooked foods at breakfast. Alcoholic beverages were limited to wine / sparking wine as it was morning. Very clean and showers are available, which was great after connecting from an overnight flight from New York. Staff were welcoming and friendly.',
 '✅ Trip Verified |  Tatty and uncared for, the BA business lounge in Gatwick South Terminal seems to be unloved and uncared for. The staff try hard but there is little loyalty (they do not work for BA)

In [26]:
all_review_text = []
text_to_replace = ['trip verified |', 'not verified |', 'unverified |', 'verified review |', '❌', '✅', '❎', '(', ')', '"', '/', '-', '.', ',', '!', '?']
for review in review_text:
    review = review.lower()
    for txt in text_to_replace:
        review = review.replace(txt, '')
    review = review.replace("'", "")
    review = review.strip()
    all_review_text.append(review)

In [27]:
all_review_text

['the lounge is clean but the seats are worn and there is not enough space to accommodate all the passengers travelling in club and those with silver and having access to the lounge nothing wrong decent sandwiches but no hot food only soup decent choice of alcoholic beverages and acceptable wines service with a smile vonly a shame the restrooms are extremely basic and not very clean',
 'the lounge is very spacious with a wide variety of areas to sit and a kids play area food was amazing with hot cooked foods at breakfast alcoholic beverages were limited to wine  sparking wine as it was morning very clean and showers are available which was great after connecting from an overnight flight from new york staff were welcoming and friendly',
 'tatty and uncared for the ba business lounge in gatwick south terminal seems to be unloved and uncared for the staff try hard but there is little loyalty they do not work for ba information on flights is scarce due to them not working direct and on one

In [28]:
reviewer_name

['E Carmere',
 'Tony Maddern',
 'S Neale',
 'Peter Glasier',
 'Richard Hodges',
 'Ellen Blaine Adams',
 'G Marsen',
 'P Horten',
 'Ian Sinclair',
 'M Kidd',
 'C Keane',
 'Jeremy Archdale',
 'C King',
 'S Keale',
 'C Mayle',
 'Jonathan Fisher',
 'J Meesen',
 'Chris Walsh',
 'Guy Senior',
 'Richard Hodges',
 'Chris Walsh',
 'Chris Walsh',
 'Jeff Michel',
 'Guy Senior',
 'Richard Hodges',
 'R Marston',
 'G Warlan',
 'Mark Ellwood',
 'Mark Ellwood',
 'Neil Jeram',
 'Mike Andrésen',
 'Mike Andrésen',
 'David Ellis',
 'P Huang',
 'Mark Ellwood',
 'R Gonzaga',
 'Sam Healey',
 'C Marvere',
 'Mark Ellwood',
 'Mark Ellwood',
 'M Neale',
 'Gabriel Pepe',
 'J Leane',
 'Alex Johnson',
 'David Ellis',
 'L Marsh',
 'D Keersen',
 'Mike Andrésen',
 'N Sareni',
 'N Barden',
 'L Xu',
 'L Xu',
 'J Vasten',
 'Richard Callis',
 'Richard Hodges',
 'Richard Hodges',
 'Mark Ellwood',
 'David Iles',
 'Neil Jeram',
 'S Meacher',
 'Roy Foster',
 'P Keane',
 'H Wealden',
 'M Bailey',
 'Derek Northcutt',
 'Andrzej 

In [29]:
date_uploaded

['14th March 2024',
 '21st December 2023',
 '28th September 2023',
 '1st September 2023',
 '20th June 2023',
 '4th May 2023',
 '28th April 2023',
 '12th April 2023',
 '11th March 2023',
 '25th January 2023',
 '17th January 2023',
 '13th December 2022',
 '10th December 2022',
 '19th November 2022',
 '24th October 2022',
 '16th October 2022',
 '25th June 2022',
 '8th June 2022',
 '21st April 2022',
 '5th April 2022',
 '5th April 2022',
 '5th April 2022',
 '4th April 2022',
 '9th February 2022',
 '15th November 2021',
 '22nd August 2021',
 '25th April 2020',
 '12th January 2020',
 '3rd January 2020',
 '17th December 2019',
 '7th December 2019',
 '7th December 2019',
 '7th December 2019',
 '20th October 2019',
 '19th October 2019',
 '16th September 2019',
 '19th August 2019',
 '18th July 2019',
 '2nd July 2019',
 '2nd July 2019',
 '3rd June 2019',
 '4th January 2019',
 '2nd January 2019',
 '19th December 2018',
 '3rd December 2018',
 '21st November 2018',
 '16th November 2018',
 '16th Nove

In [30]:
airport_names

['Barbados Grantley Adams Airport',
 'London Heathrow Airport',
 'London Gatwick Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'Manchester Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 "Chicago O'Hare Airport",
 'London Heathrow Airport',
 'Venice Marco Polo Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'Geneva Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 "Chicago O'Hare Airport",
 'Rio De Janeiro Airport',
 'London Heathrow Airport',
 'New York JFK Airport',
 'London Heathrow Airport',
 'London Heathrow Airport',
 'Johannesburg Airport',
 'Singapore Changi Airport',
 'London Heathrow Airport',
 'Washington Dulles Airport',
 'N

In [31]:
ratings_given

['6',
 '10',
 '1',
 '2',
 '9',
 '1',
 '5',
 '6',
 '2',
 '1',
 '4',
 '1',
 '7',
 '10',
 '3',
 '2',
 '9',
 '10',
 '6',
 '5',
 '7',
 '1',
 '3',
 '9',
 '6',
 '7',
 '6',
 '1',
 '7',
 '4',
 '9',
 '10',
 '7',
 '8',
 '6',
 '3',
 '9',
 '7',
 '4',
 '10',
 '6',
 '3',
 '3',
 '10',
 '5',
 '1',
 '1',
 '6',
 '6',
 '2',
 '7',
 '10',
 '4',
 '4',
 '7',
 '8',
 '3',
 '1',
 '8',
 '1',
 '6',
 '10',
 '10',
 '5',
 '4',
 '4',
 '6',
 '6',
 '3',
 '7',
 '4',
 '1',
 '4',
 '8',
 '6',
 '3',
 '6',
 '8',
 '3',
 '9',
 '1',
 '7',
 '3',
 '8',
 '2',
 '8',
 '9',
 '3',
 '3',
 '10',
 '7',
 '6',
 '7',
 '8',
 '8',
 '4',
 '8',
 '8',
 '8',
 '5',
 '6',
 '10',
 '1',
 '2',
 '9',
 '1',
 '5',
 '6',
 '2',
 '1',
 '4',
 '1',
 '7',
 '10',
 '3',
 '2',
 '9',
 '10',
 '6',
 '5',
 '7',
 '1',
 '3',
 '9',
 '6',
 '7',
 '6',
 '1',
 '7',
 '4',
 '9',
 '10',
 '7',
 '8',
 '6',
 '3',
 '9',
 '7',
 '4',
 '10',
 '6',
 '3',
 '3',
 '10',
 '5',
 '1',
 '1',
 '6',
 '6',
 '2',
 '7',
 '10',
 '4',
 '4',
 '7',
 '8',
 '3',
 '1',
 '8',
 '1',
 '6',
 '10',
 '10',
 '5

In [32]:
recommendations

['yes',
 'yes',
 'no',
 'no',
 'yes',
 'no',
 'no',
 'no',
 'no',
 'no',
 'no',
 'no',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'no',
 'yes',
 'no',
 'yes',
 'no',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'no',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'no',
 'yes',
 'no',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'no',
 'no',
 'no',
 'no',
 'no',
 'no',
 'no',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',


In [33]:
cols = ['Customer Name', 'Review Head', 'Review', 'Date', 'Airport Names', 'Ratings(10)']

In [45]:
dataset = []
for i in range(len(all_review_text)):
    data = []
    try:
        review = reviewer_name[i]
        data.append(review)
    except Exception as e:
        data.append(np.nan)
    
    try:
        sub = all_review_sub[i]
        data.append(sub)
    except Exception as e:
        data.append(np.nan)
        
    try:    
        review = all_review_text[i]
        data.append(review)
    except Exception as e:
        data.append(np.nan)
                    
    try:    
        date = date_uploaded[i]
        data.append(date)
    except:
        data.append(np.nan)
    
    try:
        airport = airport_names[i]
        data.append(airport)
    except Exception as e:
        data.append(np.nan)
    
    try:
        rating = ratings_given[i]
        data.append(rating)
    except Exception as e:
        data.append(np.nan)
        
    dataset.append(data)
        
dataset 

[['E Carmere',
  'there is not enough space',
  'the lounge is clean but the seats are worn and there is not enough space to accommodate all the passengers travelling in club and those with silver and having access to the lounge nothing wrong decent sandwiches but no hot food only soup decent choice of alcoholic beverages and acceptable wines service with a smile vonly a shame the restrooms are extremely basic and not very clean',
  '14th March 2024',
  'Barbados Grantley Adams Airport',
  '6'],
 ['Tony Maddern',
  'staff were welcoming and friendly',
  'the lounge is very spacious with a wide variety of areas to sit and a kids play area food was amazing with hot cooked foods at breakfast alcoholic beverages were limited to wine  sparking wine as it was morning very clean and showers are available which was great after connecting from an overnight flight from new york staff were welcoming and friendly',
  '21st December 2023',
  'London Heathrow Airport',
  '10'],
 ['S Neale',
  'rotte

In [46]:
temp_df = pd.DataFrame(dataset, columns = cols)

In [47]:
temp_df

,Customer Name,Review Head,Review,Date,Airport Names,Ratings(10)
0,E Carmere,there is not enough space,the lounge is clean but the seats are worn and...,14th March 2024,Barbados Grantley Adams Airport,6
1,Tony Maddern,staff were welcoming and friendly,the lounge is very spacious with a wide variet...,21st December 2023,London Heathrow Airport,10
2,S Neale,rotten and totally incompetent airline,tatty and uncared for the ba business lounge i...,28th September 2023,London Gatwick Airport,1
3,Peter Glasier,"ba staff who are rude, abrasive",crowded or rather overcrowded as usual the cat...,1st September 2023,London Heathrow Airport,2
4,Richard Hodges,full service has been resumed,flew london to kalamata and as a silversaphire...,20th June 2023,London Heathrow Airport,9
...,...,...,...,...,...,...
516,James Pemberton,it had deteriorated even further,i recently read a review on this website regar...,11th August 2009,Jersey Airport,4
517,Fabrizio Ferraro,food selection is simple,club europe lounge at gatwick is very nice it ...,30th July 2009,London Gatwick Airport,4
518,Simon Taylor,good atmosphere,small but fine for the number of travellers on...,30th July 2009,Montréal Airport,1
519,John Wilson,decor is drab,only food available was cheese and crackers a ...,15th July 2009,Antigua Vc Bird Airport,4


In [48]:
df = temp_df.copy()

In [50]:
df.to_csv('Scraped_Reviews.csv', index = False)